# Projeto - Chat Bot para pesquisa de imóveis

**Autores:** 
- Davi Reis Vieira de Souza
- Francisco Pinheiro Janela

**Disciplina:** ANÁLISE DE TEXTO DE FONTES DESESTRUTURADAS E WEB

## 1. Objetivo

Por meio da utilização da biblioteca `Beautifull Soup`, será feito um scrapping de todos os dados importantes para a construção de uma base de imóveis à venda na plataforma `ZAP Imóveis`, de forma a coletar informações como localização, número de quartos, número de banheiros, garagem, entre outras, e, por meio da integração deste resultado como *View* na API do ChatGPT, será criado um Chat Bot para realizar a querry de informações desejadas pelo usuário e filtrado na base de dados criada a resposta mais adequada.

## 2. Metodologia

Nesta etapa, será descrita a metodologia de como foi realizado o projeto, buscando detalhar o processo de coleta de informações, criação e aplicação da view no ChatGPT e de realização do *parsing* de informação para a filtragem.

### 2.1. Web Scraping

Para realizar o Web Scrapping foi utilizada a biblioteca `Beautiful Soup` que é responsável por realizar o *parsing* das *tags* **HTML** e possibilitar a busca por elas para encontrar valores específicos. Assim, ao realizar a inspeção da página e otimizar a coleta de dados para as informações relevantes, é possível montar um banco de dados completos com as informações que serão utilizadas para filtragem.

## 3. Desenvolvimento

In [1]:
## Importando Bibliotecas
# para trabalhar com diretórios / sistema operacional
import os

# para nos comunicarmos com a Web
import requests

# para extrair informações de páginas HTML
import bs4
from bs4 import BeautifulSoup

# Para expressões regulares
import re

# Para criar um Data Frame
import pandas as pd
import numpy as np

# Controlar espera entre requisições
import time

# Gerar valores aleatórios
import random

# Produto cartesiano
from sklearn.utils.extmath import cartesian

/var/folders/9t/_sm_x0m957lf5z6q_l21z3dm0000gn/T/ipykernel_20365/2020947753.py:16: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 3.1. Construindo a URL

Além de aleatorizar o `user_agent` para evitar bloqueios anti-scrapping simples, a pesquisa à plataforma será feita de forma abranjente em todas as requisições, podendo variar a cidade, o tipo de imóvel e se é aluguel ou venda. Para poder percorrer as páginas, como argumento passado na *URL* temos `pagina=`, que por meio de uma variável numériaca, será possível encontrar todas as listagens disponíveis por página.

In [2]:
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322)",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:74.0) Gecko/20100101 Firefox/74.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:72.0) Gecko/20100101 Firefox/72.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:74.0) Gecko/20100101 Firefox/74.0",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:76.0) Gecko/20100101 Firefox/76.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0",
]

In [3]:
u_agent = random.choice(user_agents)
headers = ({"User-Agent": u_agent})

In [4]:
tipoImovel = "imoveis"
cidade = "sp+sao-paulo"
aluguelOuVenda = "venda"
pagina = 1

url = f"https://www.zapimoveis.com.br/{aluguelOuVenda}/{tipoImovel}/{cidade}/?pagina={pagina}"
print(url)

https://www.zapimoveis.com.br/venda/imoveis/sp+sao-paulo/?pagina=1


### 3.2. Primeira Exploração

Utilizando as variáveis básicas apresentadas acima e olhando somente para a primeira página, serão destrinchadas as `tags` e `classes` dos componentes *HTML* da página, com o objetivo de alcançar a primeira coleta de informação.

In [5]:
resposta = requests.get(url=url, headers=headers)
resposta.encoding = "utf-8"

soup = BeautifulSoup(resposta.text, "html.parser")

anuncios = soup.find("div", {"class":"listing-wrapper"}).find_all("div", {"class":"result-card"})

print(f"Número de anúncios coletados: {len(anuncios)}")

Número de anúncios coletados: 17


In [6]:
def build_dataframe(anuncios: list[BeautifulSoup]) -> pd.DataFrame:

    cidade = []
    bairro = []
    rua = []
    descricao = []
    valor = []
    condominio = []
    iptu = []
    min_metragem = []
    max_metragem = []
    min_quartos = []
    max_quartos = []
    min_banheiros = []
    max_banheiros = []
    min_vagas = []
    max_vagas = []
    destaque = []
    super_destaque = []
    em_planta = []
    em_construcao = []
    n_anuncios = []
    link = []

    for anuncio in anuncios:
        _localizacaoAnuncio = anuncio.find("section", {"class": "card__location"})
        if _localizacaoAnuncio == None:
            continue

        # Bairro e Cidade do Imóvel
        _bairroAnuncio = _localizacaoAnuncio.find("h2")
        if _bairroAnuncio == None:
            continue
        else:
            bairro.append(_bairroAnuncio.get_text().split(", ")[0])
            cidade.append(_bairroAnuncio.get_text().split(", ")[1])

        # Rua do Imóvel
        _ruaAnuncio = _localizacaoAnuncio.find("p")
        if _ruaAnuncio == None:
            continue
        else:
            rua.append(_ruaAnuncio.get_text())

        # Descrição
        _descricaoAnuncio = anuncio.find("p", {"class": "card__description"})
        if _descricaoAnuncio == None:
            descricao.append("")
        else:
            descricao.append(_descricaoAnuncio.get_text().replace("\n", " "))

        # Detalhes do Anúncio do Imóvel
        _detalhesAnuncio = anuncio.find("div", {"class": "listing-price"})
        if _detalhesAnuncio == None:
            valor.append(-1)
            condominio.append(-1)
            iptu.append(-1)
            n_anuncios.append(-1)

        else:
            _valoresAnuncio = _detalhesAnuncio.find_all("p")
            if _valoresAnuncio == None:
                valor.append(-1)
                condominio.append(-1)
                iptu.append(-1)
            else:
                # Condominio e IPTU do Imóvel
                if (len(_valoresAnuncio) == 2):
                    if ("Cond. R$ " in _valoresAnuncio[1].get_text()):
                        _condIPTUAnuncio = _valoresAnuncio[1].get_text().split(" | ")
                        if (len(_condIPTUAnuncio) == 2):
                            iptu.append(_condIPTUAnuncio[1].replace("IPTU R$ ", ""))
                        else:
                            iptu.append(1)
                        condominio.append(_condIPTUAnuncio[0].replace("Cond. R$ ", ""))
                    else:
                        condominio.append(-1)
                        iptu.append(-1)
                else:
                    condominio.append(-1)
                    iptu.append(-1)

                # Valor do Imóvel
                _valorAnuncioTexto = _valoresAnuncio[0].get_text().replace("R$ ", "").replace(".", "").replace(",", ".").replace("\n","")
                if _valorAnuncioTexto == "Valor sob consulta":
                    valor.append(-1)
                else:
                    valor.append(_valorAnuncioTexto)

            # Número de Anúncios
            _nAnunciosAnuncio = _detalhesAnuncio.find("div", {"class": "deduplicated-complement"})
            if _nAnunciosAnuncio == None:
                n_anuncios.append(1)
            else:
                n_anuncios.append(re.findall(r'\d+',_nAnunciosAnuncio.find("p").get_text())[0])

        # Metragem do Imóvel
        _metragemAnuncio = anuncio.find('p',{'itemprop':'floorSize'})
        if _metragemAnuncio == None:
            min_metragem.append(-1)
            max_metragem.append(-1)
        else:
            _metragemAnuncioText = _metragemAnuncio.get_text().replace(" m²", "")
            if ("-" in _metragemAnuncioText):
                min_metragem.append(_metragemAnuncioText.split(" - ")[0])
                max_metragem.append(_metragemAnuncioText.split(" - ")[1])
            else:
                min_metragem.append(_metragemAnuncioText)
                max_metragem.append(_metragemAnuncioText)

        # Número de Quartos do Imóvel
        _quartosAnuncio = anuncio.find('p',{'itemprop':'numberOfRooms'})
        if _quartosAnuncio == None:
            min_quartos.append(0)
            max_quartos.append(0)
        else:
            _quartosAnuncioText = _quartosAnuncio.get_text()
            if ("-" in _quartosAnuncioText):
                min_quartos.append(_quartosAnuncioText.split(" - ")[0])
                max_quartos.append(_quartosAnuncioText.split(" - ")[1])
            else:
                min_quartos.append(_quartosAnuncioText)
                max_quartos.append(_quartosAnuncioText)

        # Número de Banheiros do Imóvel
        _banheirosAnuncio = anuncio.find('p',{'itemprop':'numberOfBathroomsTotal'})
        if _banheirosAnuncio == None:
            min_banheiros.append(1)
            max_banheiros.append(1)
        else:
            _banheirosAnuncioText = _banheirosAnuncio.get_text()
            if ("-" in _banheirosAnuncioText):
                min_banheiros.append(_banheirosAnuncioText.split(" - ")[0])
                max_banheiros.append(_banheirosAnuncioText.split(" - ")[1])
            else:
                min_banheiros.append(_banheirosAnuncioText)
                max_banheiros.append(_banheirosAnuncioText)

        # Vagas de Garagem do Imóvel
        _vagasAnuncio = anuncio.find('p',{'itemprop':'numberOfParkingSpaces'})
        if _vagasAnuncio == None:
            min_vagas.append(0)
            max_vagas.append(0)
        else:
            _vagasAnuncioText = _vagasAnuncio.get_text()
            if ("-" in _vagasAnuncioText):
                min_vagas.append(_vagasAnuncioText.split(" - ")[0])
                max_vagas.append(_vagasAnuncioText.split(" - ")[1])
            else:
                min_vagas.append(_vagasAnuncioText)
                max_vagas.append(_vagasAnuncioText)

        # Destaque ou Super Destaque
        _tagDestaqueAnuncio = anuncio.find("div", {"class": "l-card__tag--top l-card__tag--top-secondary"})
        if _tagDestaqueAnuncio == None:
            destaque.append(0)
            super_destaque.append(0)
        else:
            _tagDestaqueAnuncioText = _tagDestaqueAnuncio.find("div", {"class": "l-tag-card__content"}).get_text()
            if _tagDestaqueAnuncioText == "Destaque":
                destaque.append(1)
                super_destaque.append(0)
            elif _tagDestaqueAnuncioText == "Super Destaque":
                destaque.append(0)
                super_destaque.append(1)
            else:
                destaque.append(0)
                super_destaque.append(0)

        # Estado de Construção
        _tagConstrucaoAnuncio = anuncio.find("div", {"class": "l-tag-card l-tag-card--context-neutral l-tag-card--full-width l-card__tag--bottom"})
        if _tagConstrucaoAnuncio == None:
            em_planta.append(0)
            em_construcao.append(0)
        else:
            _tagConstrucaoAnuncioText = _tagConstrucaoAnuncio.find("div", {"class": "l-tag-card__content"}).get_text()
            if _tagConstrucaoAnuncioText == "Na planta":
                em_planta.append(1)
                em_construcao.append(0)
            elif _tagConstrucaoAnuncioText == "Em construção":
                em_planta.append(0)
                em_construcao.append(1)
            else:
                em_planta.append(0)
                em_construcao.append(0)

        # Link do Imóvel
        _linkAnuncio = anuncio.find("a")
        if _linkAnuncio == None:
            link.append("")
        else:
            link.append(_linkAnuncio['href'])


    df = pd.DataFrame({'Cidade': cidade,
                    'Bairro': bairro,
                    'Rua': bairro,
                    'Decricao': descricao,
                    'Preco': valor,
                    'Condominio': condominio,
                    'IPTU': iptu,
                    'Min. Area (m2)': min_metragem,
                    'Max. Area (m2)': max_metragem,
                    'Min. Quartos': min_quartos,
                    'Max. Quartos': max_quartos,
                    'Min. Banheiros': min_banheiros,
                    'Max. Banheiros': max_banheiros,
                    'Min. Vagas': min_vagas,
                    'Max. Vagas': max_vagas,
                    'Destaque': destaque,
                    'Super Destaque': super_destaque,
                    'Em Planta': em_planta,
                    'Em Contrucao': em_construcao,
                    'N. Anuncios': n_anuncios,
                    'Link': link
                    })

    df = df[df['Preco'] != -1]

    return df

In [7]:
df = build_dataframe(anuncios)

df.head(5)

,Cidade,Bairro,Rua,Decricao,Preco,Condominio,IPTU,Min. Area (m2),Max. Area (m2),Min. Quartos,...,Min. Banheiros,Max. Banheiros,Min. Vagas,Max. Vagas,Destaque,Super Destaque,Em Planta,Em Contrucao,N. Anuncios,Link
1,São Paulo,Santo Amaro,Santo Amaro,Em um dos bairros nobres mais tranquilos e des...,1151797,-1,-1,77,131,2,...,3,4,1,2,0,0,0,1,1,https://www.zapimoveis.com.br/lancamento/venda...
2,São Paulo,Indianópolis,Indianópolis,"Características do apartamento: - São 74m², va...",1590000,-1,-1,74,74,2,...,3,3,1,1,0,0,0,0,1,https://www.zapimoveis.com.br/lancamento/venda...
3,São Paulo,Jardim Paulista,Jardim Paulista,Flat Modus Vivendi COD 2461 A região central...,954000,1.054,199,41,41,1,...,1,1,1,1,0,1,0,0,1,https://www.zapimoveis.com.br/imovel/venda-fla...
4,São Paulo,Higienópolis,Higienópolis,"Cobertura penthouse, localizada a duas quadras...",2100000,2.150,460,130,130,2,...,3,3,1,1,0,1,0,0,1,https://www.zapimoveis.com.br/imovel/venda-apa...
5,São Paulo,Vila Firmiano Pinto,Vila Firmiano Pinto,EXCELENTE OPORTUNIDADE PRONTO PARA MORAR ALTO ...,682269,750,250,72,72,3,...,2,2,1,1,0,1,0,0,1,https://www.zapimoveis.com.br/imovel/venda-apa...


## 3.3. Recolhendo por todas as páginas

Com a coleta feita para uma página, agora é necessária uma iteração por todas as páginas do modelo de pesquisa para coletar toda a informação disponível. A medida de exemplo, abaixo será inserido um limitador de páginas, para que o arquivo salvo não ocupe muito espaço.

In [8]:
def build_url(pagina:int, tipoImovel:str="imoveis", cidade:str="sp+sao-paulo", aluguelOuVenda:str="venda") -> str:
    url = f"https://www.zapimoveis.com.br/{aluguelOuVenda}/{tipoImovel}/{cidade}/?pagina={pagina}"
    return url

def get_page(url:str) -> BeautifulSoup:
    u_agent = random.choice(user_agents)
    headers = ({"User-Agent": u_agent})

    resposta = requests.get(url=url, headers=headers)
    resposta.encoding = "utf-8"

    soup = BeautifulSoup(resposta.text, "html.parser")
    return soup

def append_to_output(df: pd.DataFrame):
    arquivo = "data/resultados.csv"
    if os.path.exists(arquivo):
        df_temp = pd.read_csv(arquivo, sep=";")
        df_final = pd.concat([df_temp, df], axis=0)
        df_final.to_csv(arquivo, index=False, sep=";")
    else:
        df.to_csv(arquivo, index=False, sep=";")


In [12]:
FINISHED = False
pagina = 1

while not FINISHED:
    url = build_url(pagina)
    page = get_page(url)

    if page == None:
        FINISHED = True
        break

    wrapper = page.find("div", {"class":"listing-wrapper"})

    if wrapper == None:
        FINISHED = True
        break
    
    anuncios = wrapper.find_all("div", {"class":"result-card"})

    if anuncios == None:
        FINISHED = True
        break

    temp_df = build_dataframe(anuncios)
    append_to_output(temp_df)
    time.sleep(2)

    pagina += 1


### 3.4. Limpeza de Dados

Depois de adquirir as informações desejadas e armazená-las em um `CSV`, agora é necessário reprocessar as categorias para garantir que os dados serão interpretados corretamente. Além disso, é necessário converter os valores **-1** das colunas de *Condominio* e *IPTU* e remover qualquer coleta que não possua área mapeada e itens duplicados que tenham sido coletados acidentalmente pelo sistema de apresentação de resultados do site.

In [13]:
df = pd.read_csv("data/resultados.csv", sep=";")
df.head(5)

,Cidade,Bairro,Rua,Decricao,Preco,Condominio,IPTU,Min. Area (m2),Max. Area (m2),Min. Quartos,...,Min. Banheiros,Max. Banheiros,Min. Vagas,Max. Vagas,Destaque,Super Destaque,Em Planta,Em Contrucao,N. Anuncios,Link
0,São Paulo,Campo Belo,Campo Belo,Próximo às estações Eucaliptos e Campo Belo do...,3100000,-1.0,-1.0,186,186,3,...,4,4,3,3,0,0,0,0,1,https://www.zapimoveis.com.br/lancamento/venda...
1,São Paulo,Santo Amaro,Santo Amaro,La Vida Estilo Barroco Seu tempo vale muito m...,532593,-1.0,-1.0,47,47,2,...,2,2,0,0,0,0,0,1,1,https://www.zapimoveis.com.br/lancamento/venda...
2,São Paulo,Vila Moraes,Vila Moraes,Localizado na Av do Cursino 3808 Stand no loca...,141587,-1.0,-1.0,24,36,1,...,1,1,0,0,0,0,1,0,1,https://www.zapimoveis.com.br/lancamento/venda...
3,São Paulo,Jardim Paulista,Jardim Paulista,Casa espaçosa com ótimo aproveitamento no Jard...,6400000,0.0,25.48,360,360,3,...,3,3,6,6,0,1,0,0,1,https://www.zapimoveis.com.br/imovel/venda-cas...
4,São Paulo,Água Branca,Água Branca,Encante-se com este lindo apartamento no bairr...,240000,340.0,0.0,31,31,1,...,1,1,0,0,0,1,0,0,1,https://www.zapimoveis.com.br/imovel/venda-apa...


In [14]:
df.dtypes

Cidade             object
Bairro             object
Rua                object
Decricao           object
Preco               int64
Condominio        float64
IPTU               object
Min. Area (m2)      int64
Max. Area (m2)      int64
Min. Quartos        int64
Max. Quartos        int64
Min. Banheiros      int64
Max. Banheiros      int64
Min. Vagas          int64
Max. Vagas          int64
Destaque            int64
Super Destaque      int64
Em Planta           int64
Em Contrucao        int64
N. Anuncios         int64
Link               object
dtype: object

In [15]:
# Substituir a categoria dos dados destaque, super destaque, em planta e em construção para boleano, mapeando 0 para False e 1 para True
df['Destaque'] = df['Destaque'].astype(bool)
df['Super Destaque'] = df['Super Destaque'].astype(bool)
df['Em Planta'] = df['Em Planta'].astype(bool)
df['Em Contrucao'] = df['Em Contrucao'].astype(bool)

df.dtypes

Cidade             object
Bairro             object
Rua                object
Decricao           object
Preco               int64
Condominio        float64
IPTU               object
Min. Area (m2)      int64
Max. Area (m2)      int64
Min. Quartos        int64
Max. Quartos        int64
Min. Banheiros      int64
Max. Banheiros      int64
Min. Vagas          int64
Max. Vagas          int64
Destaque             bool
Super Destaque       bool
Em Planta            bool
Em Contrucao         bool
N. Anuncios         int64
Link               object
dtype: object

In [16]:
df.head(5)

,Cidade,Bairro,Rua,Decricao,Preco,Condominio,IPTU,Min. Area (m2),Max. Area (m2),Min. Quartos,...,Min. Banheiros,Max. Banheiros,Min. Vagas,Max. Vagas,Destaque,Super Destaque,Em Planta,Em Contrucao,N. Anuncios,Link
0,São Paulo,Campo Belo,Campo Belo,Próximo às estações Eucaliptos e Campo Belo do...,3100000,-1.0,-1.0,186,186,3,...,4,4,3,3,False,False,False,False,1,https://www.zapimoveis.com.br/lancamento/venda...
1,São Paulo,Santo Amaro,Santo Amaro,La Vida Estilo Barroco Seu tempo vale muito m...,532593,-1.0,-1.0,47,47,2,...,2,2,0,0,False,False,False,True,1,https://www.zapimoveis.com.br/lancamento/venda...
2,São Paulo,Vila Moraes,Vila Moraes,Localizado na Av do Cursino 3808 Stand no loca...,141587,-1.0,-1.0,24,36,1,...,1,1,0,0,False,False,True,False,1,https://www.zapimoveis.com.br/lancamento/venda...
3,São Paulo,Jardim Paulista,Jardim Paulista,Casa espaçosa com ótimo aproveitamento no Jard...,6400000,0.0,25.48,360,360,3,...,3,3,6,6,False,True,False,False,1,https://www.zapimoveis.com.br/imovel/venda-cas...
4,São Paulo,Água Branca,Água Branca,Encante-se com este lindo apartamento no bairr...,240000,340.0,0.0,31,31,1,...,1,1,0,0,False,True,False,False,1,https://www.zapimoveis.com.br/imovel/venda-apa...


In [17]:
# Substituir valores -1 por nulo
df['Condominio'] = df['Condominio'].replace(-1, None)
df['IPTU'] = df['IPTU'].replace(-1, None)

df.head(5)

,Cidade,Bairro,Rua,Decricao,Preco,Condominio,IPTU,Min. Area (m2),Max. Area (m2),Min. Quartos,...,Min. Banheiros,Max. Banheiros,Min. Vagas,Max. Vagas,Destaque,Super Destaque,Em Planta,Em Contrucao,N. Anuncios,Link
0,São Paulo,Campo Belo,Campo Belo,Próximo às estações Eucaliptos e Campo Belo do...,3100000,None,-1.0,186,186,3,...,4,4,3,3,False,False,False,False,1,https://www.zapimoveis.com.br/lancamento/venda...
1,São Paulo,Santo Amaro,Santo Amaro,La Vida Estilo Barroco Seu tempo vale muito m...,532593,None,-1.0,47,47,2,...,2,2,0,0,False,False,False,True,1,https://www.zapimoveis.com.br/lancamento/venda...
2,São Paulo,Vila Moraes,Vila Moraes,Localizado na Av do Cursino 3808 Stand no loca...,141587,None,-1.0,24,36,1,...,1,1,0,0,False,False,True,False,1,https://www.zapimoveis.com.br/lancamento/venda...
3,São Paulo,Jardim Paulista,Jardim Paulista,Casa espaçosa com ótimo aproveitamento no Jard...,6400000,0.0,25.48,360,360,3,...,3,3,6,6,False,True,False,False,1,https://www.zapimoveis.com.br/imovel/venda-cas...
4,São Paulo,Água Branca,Água Branca,Encante-se com este lindo apartamento no bairr...,240000,340.0,0.0,31,31,1,...,1,1,0,0,False,True,False,False,1,https://www.zapimoveis.com.br/imovel/venda-apa...


In [18]:
print(f"Quantidade de registros: {df.shape[0]}")

# Eliminar duplicatas
df = df.drop_duplicates()

print(f"Quantidade de registros depois de eliminar duplicatas: {df.shape[0]}")

Quantidade de registros: 5285
Quantidade de registros depois de eliminar duplicatas: 4293


In [19]:
# Salvando o DataFrame em um arquivo CSV limpo
df.to_csv("data/resultados_limpos.csv", index=False, sep=";")

### 3.5. Iteração da coleta

Como foi identificado, os dados coletados não possuem o número do imóvel na rua, apenas se a página do anúncio for acessada. Portanto, a coleta será feita acessando o link de cada anúncio e criando uma nova linha com esse valor.

In [4]:
df = pd.read_csv("data/resultados_limpos.csv", sep=";")

In [5]:
def get_number(link_anuncio: str) -> int:
    if link_anuncio == np.nan or link_anuncio == None or link_anuncio == "":
        return -1

    try:
        u_agent = random.choice(user_agents)
        headers = ({"User-Agent": u_agent})

        resposta = requests.get(url=link_anuncio, headers=headers)
        resposta.encoding = "utf-8"
        
        soup = BeautifulSoup(resposta.text, "html.parser")

        _adressAnuncio = soup.find("div", {"class": "address-info-wrapper"})
        if _adressAnuncio == None:
            return -1
        else:
            _numeroAnuncio = _adressAnuncio.find_all("p")
            if _numeroAnuncio == None:
                return -1
            else:
                return _numeroAnuncio[0].get_text().replace(" ", "").split(",")[1].split("-")[0]
    except:
        return -1

In [6]:
print(get_number("https://www.zapimoveis.com.br/lancamento/venda-apartamento-3-quartos-campo-belo-zona-sul-sao-paulo-sp-186m2-id-2587491017/"))

635


In [7]:
# using the column link, run the function get_number to get the price of the property
df['Numero'] = df['Link'].apply(get_number)

In [8]:
#reordenar as colunas para que a coluna Numero fique na quarta posição, depois de Rua
df = df[['Cidade', 'Bairro', 'Rua', 'Numero', 'Decricao', 'Preco', 'Condominio', 'IPTU', 'Min. Area (m2)', 'Max. Area (m2)', 'Min. Quartos', 'Max. Quartos', 'Min. Banheiros', 'Max. Banheiros', 'Min. Vagas', 'Max. Vagas', 'Destaque', 'Super Destaque', 'Em Planta', 'Em Contrucao', 'N. Anuncios', 'Link']]
df.head(5)

,Cidade,Bairro,Rua,Numero,Decricao,Preco,Condominio,IPTU,Min. Area (m2),Max. Area (m2),...,Min. Banheiros,Max. Banheiros,Min. Vagas,Max. Vagas,Destaque,Super Destaque,Em Planta,Em Contrucao,N. Anuncios,Link
0,São Paulo,Campo Belo,Campo Belo,635,Próximo às estações Eucaliptos e Campo Belo do...,3100000,NaN,-1.0,186,186,...,4,4,3,3,False,False,False,False,1,https://www.zapimoveis.com.br/lancamento/venda...
1,São Paulo,Santo Amaro,Santo Amaro,633,La Vida Estilo Barroco Seu tempo vale muito m...,532593,NaN,-1.0,47,47,...,2,2,0,0,False,False,False,True,1,https://www.zapimoveis.com.br/lancamento/venda...
2,São Paulo,Vila Moraes,Vila Moraes,3908,Localizado na Av do Cursino 3808 Stand no loca...,141587,NaN,-1.0,24,36,...,1,1,0,0,False,False,True,False,1,https://www.zapimoveis.com.br/lancamento/venda...
3,São Paulo,Jardim Paulista,Jardim Paulista,SãoPaulo,Casa espaçosa com ótimo aproveitamento no Jard...,6400000,0.0,25.48,360,360,...,3,3,6,6,False,True,False,False,1,https://www.zapimoveis.com.br/imovel/venda-cas...
4,São Paulo,Água Branca,Água Branca,409,Encante-se com este lindo apartamento no bairr...,240000,340.0,0.0,31,31,...,1,1,0,0,False,True,False,False,1,https://www.zapimoveis.com.br/imovel/venda-apa...


In [9]:
# Limpar tudo o que não for numero ou -1 e substituir por nulo da coluna de numeros
df['Numero'] = df['Numero'].replace(regex=r'[^0-9-]', value='', inplace=False)

In [10]:
# salvar em um arquivo CSV
df.to_csv("data/resultados_completos.csv", index=False, sep=";")